# Generates Table of detection probabilities and key metrics for the CPP Targets Wiki

### Imports

In [ ]:
import os, numpy as np, pandas as pd
from astropy.time import Time
from radvel.basis import Basis
from radvel.utils import Msini
from orbitize.basis import tp_to_tau
from orbitize.kepler import calc_orbit
from astropy import units as u, constants as c
from pathlib import Path
import matplotlib.pyplot as plt
from roman_pointing.roman_pointing import calcRomanAngles, getL2Positions
from astroquery.simbad import Simbad
from astropy.coordinates import (
    SkyCoord,
    Distance,
    BarycentricMeanEcliptic,
)

from roman_table import *

### Helper Functions

In [ ]:
def is_detectable(seps,fc,contrast_curve):
    """
    TODO:
    - Make sure IWA and OWA are handled appropriately
    - Think about linear interpolation for v2
    """
    seps = np.array(seps)
    fc = np.array(fc)

    # get the position of the closest separation in the contrast curve for each given sep
    concurve_seps = contrast_curve[0]
    concurve_fcs = contrast_curve[1]
    args = np.argmin(np.abs(seps[:,np.newaxis]-concurve_seps),axis=1)
    limiting_fcs = concurve_fcs[args]
    return fc >= limiting_fcs

def test_is_detectable():
    test_seps = [4,   5,   6,   0.5, 50]
    test_fcs =  [1e-4,1e-5,1e-6,1e-1,1e-1]
    
    test_concurve = np.array([
        [1,5,   10,  15,  20,  25,  30,  40], # sep_mas
        [1,1e-5,1e-6,1e-7,1e-7,1e-7,1e-7,1] # contrast limit
    ])

    expected_out = np.array([ True,  True, False, False, False])

    assert np.all(expected_out == is_detectable(test_seps,test_fcs,test_concurve))

test_is_detectable()

### Main

In [ ]:
# Load contrast curve
concurve_fpath = '/Users/sbogat/Documents/01_Research/exoplanner_workspace/data/concurves/concurve_optimistic.csv'
concurve_df = pd.read_csv(concurve_fpath)
concurve_df['D'] = 2.4
concurve_df['sep_mas'] = concurve_df.loD * concurve_df['lambda'] * 1e-9 / concurve_df['D'] * u.radian.to(u.mas)
display(concurve_df)
concurve = np.array([
    concurve_df.sep_mas.values,
    concurve_df.contr.values])

In [ ]:
planets = list(display_names.keys())
start_date, end_date = "2027-01-01", "2028-06-01"
plot=True
orbit_props_dir = '/Users/sbogat/Documents/01_Research/exoplanner_workspace/output/orbit_props'

# Temporary while we wait for albedos from Dmitry
albedo_med = 0.4
albedo_std = 0.1

for p,planet in enumerate(planets):
    print(f'Planet index {p}/{len(planets)-1}: {planet}')

    input_fpath = os.path.join(
        orbit_props_dir,
        f'{planet}_{start_date}_to_{end_date}_RVOnly.csv'
    )

    output_fname = f'{planet}_{start_date}_to_{end_date}_RVOnly_DetectabilitySummary.csv'

    # GET INFO FROM THE HEADER
    with open(input_fpath,'r') as f:
        header_lines = []
        for l,line in enumerate(f):
            if line.startswith('#'): 
                # print(l,line,end='')
                header_lines.append(line)
            else:
                break

        pl_mass = float(header_lines[11][15:20])
        pl_mass_upper = float(header_lines[11][22:27])
        pl_mass_lower = float(header_lines[11][29:34])

        pl_rad = float(header_lines[12][17:22])
        pl_rad_upper = float(header_lines[12][24:29])
        pl_rad_lower = float(header_lines[12][31:36])


    print(f'.   Planet mass: {pl_mass} +{pl_mass_upper} -{pl_mass_lower} M_Jup')
    print(f'.   Planet radius: {pl_rad} +{pl_rad_upper} -{pl_rad_lower} R_Jup')

    df_in = pd.read_csv(input_fpath,skiprows=15,index_col='date_iso')

    # Calculate flux contrast
    albedos = np.random.normal(albedo_med,albedo_std,size=len(df_in))
    # TODO: Update to use albedos from dmitry, orbital rad instead of sep
    df_in['fc_med'] = albedos * df_in.lambert_phase_median * (pl_rad*u.R_jup.to(u.AU) / df_in.separation_au_median)**2
        
    # Calculate BG observation windows
    simbad = Simbad()
    simbad.add_votable_fields("pmra", "pmdec", "plx_value", "rvz_radvel")
    ts = Time(list(df_in.index))
    res = simbad.query_object("Sagittarius A*")
    gb = SkyCoord(
        res["ra"].value.data[0],
        res["dec"].value.data[0],
        unit=(res["ra"].unit, res["dec"].unit),
        frame="icrs",
        distance=Distance(8 * u.kpc),
        pm_ra_cosdec=0 * res["pmra"].unit,
        pm_dec=0 * res["pmdec"].unit,
        radial_velocity=0 * res["rvz_radvel"].unit,
        equinox="J2000",
        obstime="J2000",
    ).transform_to(BarycentricMeanEcliptic)
    sun_ang_ref, yaw_ref, pitch_ref, B_C_I_targ = calcRomanAngles(
        gb, ts, getL2Positions(ts)
    )
    df_in['GB_not_observable'] = ~((sun_ang_ref.to_value(u.deg) > 54) & (sun_ang_ref.to_value(u.deg) < 126))
    
    # Calculate target observability windows
    simbad_name = " ".join(planet.split("_"))
    res = simbad.query_object(simbad_name)
    target = SkyCoord(
        res["ra"].value.data[0],
        res["dec"].value.data[0],
        unit=(res["ra"].unit, res["dec"].unit),
        frame="icrs",
        distance=Distance(parallax=res["plx_value"].value.data[0] * res["plx_value"].unit),
        pm_ra_cosdec=res["pmra"].value.data[0] * res["pmra"].unit,
        pm_dec=res["pmdec"].value.data[0] * res["pmdec"].unit,
        radial_velocity=res["rvz_radvel"].value.data[0] * res["rvz_radvel"].unit,
        equinox="J2000",
        obstime="J2000",
    ).transform_to(BarycentricMeanEcliptic)

    sun_ang_targ, yaw_targ, pitch_targ, B_C_I_targ = calcRomanAngles(
        target, ts, getL2Positions(ts)
    )
    df_in['targ_observable'] = (sun_ang_targ.to_value(u.deg) > 54) & (sun_ang_targ.to_value(u.deg) < 126)    

    # Calculate detection probability
    # TODO:
    # - generate spread of separations, flux contrasts based on med and uncertainty of each parameter at each time
    # - count percentage of generated points are detectable


    display(df_in)

    break # Do 1 planet at a time while developing


